## <b>■ 복습</b>
    1장
    2장
    3장
    4장
    5장
    6장
    7장. CNN + Fully Connected
        Image -----> Convolution ----------------> Pooling ----------------> Fully Connected
                     여러개의 Feature map           Feature map 선명하게             분류
                     이미지의 특징 추출
                              ↓                            ↓                       ↓
                     Convolution class 생성         Pooling class 생성        5장 오차역전파 3층 신경망  
    
    8장. 딥러닝의 역사
![hist](http://cfile252.uf.daum.net/image/992CCB4F5F3472F40F4BEF)
![hist2](http://cfile253.uf.daum.net/image/9969A7455F3473150F3FDF)

## <b>■ 밑바닥 딥러닝 책 최종 스크립트의 큰 그림</b>
### <b>□ 필기체 데이터를 CNN 신경망에 넣어서 학습시키는 그림</b>
![learn1](learn1.png)
![learn2](http://cfile249.uf.daum.net/image/99BA49355F3465D00CD4EA)

<b>첫번째 그림</b>
#### 예제1. 28x28 필기체 데이터 100장을 생성

In [8]:
import numpy as np
x = np.random.randn(100,1,28,28)
print(x.shape)

(100, 1, 28, 28)


#### 예제2. 5x5 필터 30장 생성

In [3]:
w1 = np.random.randn(30,1,5,5)
print(w1.shape)

(30, 1, 5, 5)


#### 예제3. 위에서 만든 입력값 x와 필터 w1을 합성곱한 feature map을 생성하고 shape를 확인하시오

In [4]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2 * pad - filter_h) // stride + 1
    out_w = (W + 2 * pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return col

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1+ (H+ 2*self.pad - FH) / self.stride)
        out_w = int(1+ (W+ 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out

In [10]:
b1 = 1
con = Convolution(w1, b1)
feature_map = con.forward(x)
print(feature_map.shape)

(100, 30, 24, 24)


#### 예제4. Convolution 계층을 통과한 output을 pooling 계층에 통과시키고 나온 output 사이즈를 확인하시오

In [11]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

In [12]:
pool = Pooling(2,2,stride=2)
pool_out = pool.forward(feature_map)
print(pool_out.shape)

(100, 30, 12, 12)


#### 예제5. 위의 pooling계층을 통과한 출력 데이터가 완전 연결계층에 들어가서 가중치와 내적할 수 있도록 가중치 w2를 생성하시오

In [14]:
w2 = np.random.randn(30*12*12, 100)

#### 예제6. 완전 연결계층의 w2와 내적한 후 더하기 할 b2를 생성하시오

In [13]:
b2 = np.zeros(100)

#### 예제7. 완전연결 계층의 마지막 출력층인 w3와 b3를 각각 생성하시오

In [17]:
w3 = np.random.randn(100,10)
b3 = np.zeros(10)
print(w3.shape)

(100, 10)


#### 예제8. pooling층을 통과한 pool_out이 완전연결계층으로 들어가서 나온 결과 shape를 출력하시오

In [19]:
pool_out2 = pool_out.reshape(100,-1)
fully_out1 = np.dot(pool_out2, w2) + b2
fully_out2 = np.dot(fully_out1, w3) + b3
print(fully_out2.shape)

(100, 10)
